In [124]:
import pandas as pd
import re
import numpy as np
import openpyxl

import sys
sys.path.append('../../zoho_data_app/')

import main_module as m
import zoho_filter_module as fil

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
path = r'C:\Users\fajar\Documents\Python\Data\Arrival List Raw\BG WESTERN ARRIVAL LIST 2022.xlsx'
xl = pd.ExcelFile(path)
sheets = xl.sheet_names  # see all sheet names
sheets

['OPEN DATE',
 'PENDING',
 'Cancel Move As Open Date Bkg',
 'CANCEL',
 'PRE ARRIVAL SUMMARY 2023',
 'Dec 23',
 'Oct 23',
 'Sep 23',
 'Aug 23',
 'July 23',
 'June 23',
 'May 23',
 'Apr 23',
 'Mar 23',
 'Feb 23',
 'JAN 23',
 'PRE ARRIVAL SUMMARY 2022',
 'Dec 22',
 'Nov 22',
 'Oct 22',
 'Sep 22',
 'Aug 22',
 'July 22',
 'Jun 22']

In [126]:
delete_sheets = ['OPEN DATE',
 'PENDING',
 'Cancel Move As Open Date Bkg',
 'CANCEL',
 'PRE ARRIVAL SUMMARY 2023',
 'PRE ARRIVAL SUMMARY 2022']

arr = np.array(sheets)
# arr = np.delete(arr, delete_sheets)

sheet_will_be_used = []

for i in sheets:
    sheets_name = i
    result = np.isin(sheets_name, delete_sheets)
    if result == True:
        print(sheets_name)
    else:
        print(sheets_name)
        sheet_will_be_used.extend([sheets_name])

sheet_will_be_used

# arr

OPEN DATE
PENDING
Cancel Move As Open Date Bkg
CANCEL
PRE ARRIVAL SUMMARY 2023
Dec 23
Oct 23
Sep 23
Aug 23
July 23
June 23
May 23
Apr 23
Mar 23
Feb 23
JAN 23
PRE ARRIVAL SUMMARY 2022
Dec 22
Nov 22
Oct 22
Sep 22
Aug 22
July 22
Jun 22


['Dec 23',
 'Oct 23',
 'Sep 23',
 'Aug 23',
 'July 23',
 'June 23',
 'May 23',
 'Apr 23',
 'Mar 23',
 'Feb 23',
 'JAN 23',
 'Dec 22',
 'Nov 22',
 'Oct 22',
 'Sep 22',
 'Aug 22',
 'July 22',
 'Jun 22']

In [127]:
def load_data_multiple(dir_path, sheet_will_be_used, cols):

    import os

    count = 0
    frames = []

    for i in sheet_will_be_used:
        print('Compiling sheets '+i+' On progress')
        count += 1
        globals()['df%s' % count] = pd.read_excel(path, dtype='string', sheet_name=i, index_col=None, header=None)
        globals()['new_df_start_segment%s' % count] = globals()['df%s' % count].loc[globals()['df%s' % count][0] == 'No'].index.values.astype(int)[0]
        globals()['new_df_end_segment%s' % count] = globals()['df%s' % count].shape[0]

        globals()['new_df%s' % count] = globals()['df%s' % count].iloc[globals()['new_df_start_segment%s' % count]:globals()['new_df_end_segment%s' % count]].reset_index().copy()
        globals()['new_df%s' % count].columns = globals()['new_df%s' % count].iloc[0]
        globals()['new_df%s' % count].drop([0,0], inplace=True)
        globals()['arr_df%s' % count] = globals()['new_df%s' % count][cols].loc[~globals()['new_df%s' % count]['No'].isna()].copy()
        globals()['arr_df%s' % count].reset_index(inplace=True)
        globals()['arr_df%s' % count].drop(globals()['arr_df%s' % count].columns[[0]],axis = 1, inplace=True)
        globals()['arr_df%s' % count]['Sheet Name'] = i
        frames.append(globals()['arr_df%s' % count])

    result = pd.concat(frames, ignore_index=True)
    return result

In [128]:
cols = ['No', 
        'Programme', 
        'Nationality',
        'Region',
        'Client Name', 
        'Email', 
        'Phone',
        'Check in',
        'Check out']

df = load_data_multiple(path, sheet_will_be_used, cols)
df

Compiling sheets Dec 23 On progress
Compiling sheets Oct 23 On progress
Compiling sheets Sep 23 On progress
Compiling sheets Aug 23 On progress
Compiling sheets July 23 On progress
Compiling sheets June 23 On progress
Compiling sheets May 23 On progress
Compiling sheets Apr 23 On progress
Compiling sheets Mar 23 On progress
Compiling sheets Feb 23 On progress
Compiling sheets JAN 23 On progress
Compiling sheets Dec 22 On progress
Compiling sheets Nov 22 On progress
Compiling sheets Oct 22 On progress
Compiling sheets Sep 22 On progress
Compiling sheets Aug 22 On progress
Compiling sheets July 22 On progress
Compiling sheets Jun 22 On progress


,No,Programme,Nationality,Region,Client Name,Email,Phone,Check in,Check out,Sheet Name
0,1,EDM,Australian,Australia,Ian Hall & Catherine Hall,cathieh4@bigpond.com,61351541496,2023-12-23 00:00:00,2023-12-30 00:00:00,Dec 23
1,1,UTBOUND SALES BOOKING,Nigerian,Australia,Mr/s. Moses Omogbeme & Pamela Omogbeme,mosesomogbeme@gmail.com,61420357974,2023-10-01 00:00:00,2023-10-06 00:00:00,Oct 23
2,2,<NA>,Australian,Australia,Lovina Te_Arani Shorland & Julius Mika,lovinashortland@yahoo.co.nz,61412279500,2023-10-08 00:00:00,2023-10-15 00:00:00,Oct 23
3,1,Facebook,Australian,Australia,Peter Roy Vardy & Rena Vardy,petervardy3@gmail.com,61415335259,2023-09-15 00:00:00,2023-09-22 00:00:00,Sep 23
4,1,BGAU FBCOMP BOOKING,Australian,Australia,Mr/s. Eva Erika Petersen & Christian Petersen,petersensgarage021@gmail.com,61477002042,2023-08-01 00:00:00,2023-08-06 00:00:00,Aug 23
...,...,...,...,...,...,...,...,...,...,...
183,2,Facebook Comp,German / Indonesian,German,Lothar Heinze & Priscilla Heinze,priscilla.heinze@gmail.com,491623885499,2022-07-11 00:00:00,2022-07-20 00:00:00,July 22
184,3,Competition,Austrlian,Australia,Leslie Follent & Carmel Follent,lfollent@gmail.com,61447310764,2022-07-12 00:00:00,2022-07-17 00:00:00,July 22
185,1,Facebook Inbound,Australian,Australia,Pasquale Micelotta & Dianne Micelotta,enaka22@live.com.au,61417011117,2022-06-01 00:00:00,2022-06-06 00:00:00,Jun 22
186,2,Facebook,South African,Australia,Douglas F Vinnicombe & Leoni Vinnicombe,leonivinn@outlook.com,61452289410,2022-06-12 00:00:00,2022-06-17 00:00:00,Jun 22


In [129]:
# df = pd.read_excel(path, dtype='string', sheet_name='Nov 22', index_col=None, header=None)
# df

In [130]:
# # mencari nomor row index untuk memulai compile data

# start_segment = df.loc[df[0] == 'No'].index.values.astype(int)[0]
# end_segment = df.shape[0]

# new_df = df.iloc[start_segment:end_segment].reset_index().copy()
# new_df.columns = new_df.iloc[0] # define kolom berdasarkan row paling awal
# new_df.drop([0,0], inplace=True) # menghapus row yang sudah dijadikan kolom (karena meskipun row index 0 sudah di define sebagai kolom, row index 0 tetap ada di row record data)
# arr_df = new_df[[
#     'NO.', 
#     'Programme', 
#     'Client Name', 
#     'Email Address', 
#     'Phone Number',
#     'Check in',
#     'Check out']].loc[~new_df['No'].isna()].copy()
# arr_df.reset_index(inplace=True)
# arr_df.drop(arr_df.columns[[0]],axis = 1, inplace=True)
# arr_df

In [131]:
df.loc[~df['Check in'].str.contains('00:00:00')]

,No,Programme,Nationality,Region,Client Name,Email,Phone,Check in,Check out,Sheet Name


In [132]:
df.rename(columns = {'No':'Sheet Row No.'}, inplace = True)

In [133]:
df.to_excel(r'C:\Users\fajar\Documents\Python\Data\Arrival List Raw\Compile Arrival\BG WESTERN ARRIVAL LIST 2022-compile.xlsx', index=False)